# POC Power BI TS Formation

In [177]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from os import path
import glob
import xlsxwriter
import openpyxl 

## 1) Déclaration des fonctions exploitées dans le code

### Génération/Mise à jour du fichier setting

In [178]:
def import_data():
    all_dir = glob.glob("Data/*.csv")    
    all_files_name = [x[5:] for x in all_dir]
    all_files_name = [x[:-4] for x in all_files_name]
    df_files_col = pd.DataFrame()
    p = 1
    
    for dir in all_dir:
        df = pd.read_csv(dir, encoding='utf-8', sep=';')
        
        for i, col in enumerate(df.columns):
            df_files_col.at[p, 'Nom fichier source'] = dir[5:-4]
            df_files_col.at[p,'Nom champ source'] = col
            df_files_col.at[p,'Type'] = str(df[col].dtypes)
            df_files_col.at[p,'Synthèse'] = str(df[col].describe())
            df_files_col.at[p, 'Répertoire'] = dir
            p = p + i
    
    df_dir_files = df_files_col.copy()
    df_dir_files = df_dir_files[["Nom fichier source", "Répertoire"]].drop_duplicates()
    
    if path.exists("Settings & documentation\Settings.xlsx"):
        pass

    else:

        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='xlsxwriter')
        workbook  = writer.book
        df_dir_files.to_excel(writer, sheet_name='Fichiers Source', index=False)
        
        df_files_col[['Nom fichier source', 'Nom champ source', 'Type', 'Synthèse' ]].to_excel(writer, sheet_name='Fichiers et colonnes source', index=False)
                
        worksheet_FCS = writer.sheets["Fichiers et colonnes source"]
        worksheet_FS = writer.sheets["Fichiers Source"]
        worksheet1 = workbook.add_worksheet("Mapping données")
        
        cell_format_FCS = workbook.add_format() 
        cell_format_FCS.set_text_wrap()
        cell_format_FCS.set_align('center')
        cell_format_FCS.set_align('vcenter')
        
        cell_format_FS = workbook.add_format()    
        cell_format_FS.set_align('left')
                
        worksheet_FCS.set_column('A:B', 30, cell_format_FCS)
        worksheet_FCS.set_column('C:C', 15, cell_format_FCS)
        worksheet_FCS.set_column('D:D', 25, cell_format_FCS)
        worksheet_FS.set_column('A:A', 40, cell_format_FS)
        worksheet_FS.set_column('B:B', 60, cell_format_FS)

        writer.save()
        workbook.close()

# Fonction créant la feuille Mapping dim dans le fichier Settings
# Cette feuille contient l'ensemble des valeurs de dimension et permet de réaliser un mapping pour changer le nom des attributs

def add_sheet_mapcol(DicDataframe):
    
    df = pd.DataFrame(columns=["Nom dimension", "Nom colonne"],data=[])
    
    for key, value in DicDataframe.items():        
        if len(key) >= 3:
            if key[:3] == "dim":
                for i, col in enumerate(value.columns):
                    df.at[i, "Nom dimension"] = key
                    df.at[i, "Nom colonne"] = col
                    df.at[i, "Nouveau_nom"] = ""
                    df.at[i, "A mapper"] = "Non"                
      
    workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
    writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
    writer.book = workbook1
    df.to_excel(writer, sheet_name="Mapping dim colonne",engine='openpyxl',index=False)     
    writer.save()
    writer.close()
    
    workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")
    writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
    writer.book = workbook1
    workbook1["Mapping dim colonne"].column_dimensions["A"].width = 20
    workbook1["Mapping dim colonne"].column_dimensions["B"].width = 20
    workbook1["Mapping dim colonne"].column_dimensions["C"].width = 20
    
    writer.save()
    writer.close()
    
import_data()

### Fonction mapping des noms de colonnes

In [179]:
# Fonction transformant le nom des colonnes suivant le mapping effectué dans l'onglet Mapping dim colonne
# Entrée: Dictionnaire key: Nom dataframe  Value : Dataframe
# Sortie un dictionnaire "nom"/Dataframe avec les valeurs des colonnes actualisées suivant l'onglet de mapping "Mapping dim colonne" du fichier settings

def map_col(dict_dataframe):
    md = pd.read_excel(r"Settings & documentation\Settings.xlsx", sheet_name="Mapping dim colonne")
    mdf = md["Nom dimension"].unique()
    dic_map = {}
    for dim in mdf:
        map = md.loc[(md["Nom dimension"] == dim ) & (md["Nouveau nom"].notnull()== True),["Nom colonne","Nouveau nom"]]
        map.reset_index(drop=True, inplace=True)
            
        for i, val in enumerate(map.iterrows()):           
            dict_dataframe[dim].rename(columns={map.at[i, "Nom colonne"]:map.at[i,"Nouveau nom"]},inplace=True)
            
    return dict_dataframe            

In [180]:
def mapp_data(dict_dataframe):
    to_mapp_data = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping données")

    if to_mapp_data.empty == True:
        mapp_col = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping dim colonne")
        mapp_col = mapp_col.loc[mapp_col["A mapper"] == "Oui", ["Nom dimension", "Nom colonne", "Nouveau nom"]]
        mapp_col["Colonne"] = ""
        mapp_col.reset_index(inplace=True, drop=True)
        

        if mapp_col.empty == False:
            for i, val in enumerate(mapp_col.iterrows()):
                if str(mapp_col.at[i, "Nouveau nom"]) != "nan":
                    mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nouveau nom"]
                else:
                     mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nom colonne"]
           
            for i,val in enumerate(mapp_col[["Nom dimension", "Colonne"]].iterrows()):
                if to_mapp_data.empty == True:
                    if (dict_dataframe[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).empty == False:
                        #print(mapp_col.at[i, "Nom dimension"])
                        #print(dict_dataframe[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]])
                        #(dict_dataframe[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).unique()
                        list_val = (dict_dataframe[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).unique()
                        #val_2 = [x for x in val if str(x) != 'nan']
                        #list_val = val_2
                        
                        print(list_val)
                        #print("hfhfjfdf")


## 2) Import des données

In [181]:
%%time
### Ajouter les tables à intégrer ici
tcw = pd.read_csv(r'Data/TrainingCollectiveWishes.csv', engine=None,encoding='utf-8', sep=';') # Demandes collectives non affectées à un plan de formation
tiw = pd.read_csv(r'Data/TrainingIndividualWishes.csv', engine=None,encoding='utf-8', sep=';') # Demandes individuelles non affectées à un plan de formation
tp = pd.read_csv(r'Data/TrainingPlan.csv', engine=None,encoding='utf-8', sep=';') # Plans de formation
tpcw = pd.read_csv(r'Data/TrainingPlanCollectiveWishes.csv', engine=None,encoding='utf-8', sep=';') # Demande de formation collectives prises en charge dans un plan de formation
tpci = pd.read_csv(r'Data/TrainingPlanIndividualWishes.csv', engine=None,encoding='utf-8', sep=';') # Demande de formation individuelles prises en charge dans un plan de formation
tr = pd.read_csv(r'Data/TrainingRegister.csv', engine=None,encoding='utf-8', sep=';') # Table recensant les inscription aux sessiosn de formation 
ts = pd.read_csv(r'Data/TrainingSessions.csv', engine=None,encoding='utf-8', sep=';') # Table recensant les sessions de formation 
tsc = pd.read_csv(r'Data/TrainingStageCost.csv', engine=None,encoding='utf-8', sep=';') # Table contenant les coûts des stages
tsv2 = pd.read_csv(r'Data/TrainingStagev2.csv', engine=None,encoding='utf-8', sep=';') # Table contenant les stages
emp = pd.read_csv(r'Data/Employees.csv', engine=None,encoding='utf-8', sep=';') # Table maître employé
empc = pd.read_csv(r'Data/EmployementContract.csv', engine=None,encoding='utf-8', sep=';') # Table contrat employé
indO = pd.read_csv(r'Data/IndividualOrganization.csv', engine=None,encoding='utf-8', sep=';') # Table organisation employé
indpp = pd.read_csv(r'Data/IndivPPCsNew.csv', engine=None,encoding='utf-8', sep=';') # ?
#setting = pd.read_excel(r'Data/Settings.xlsx', sheets="Mapp_data") # Table de paramétrage servant à mapper les données

Wall time: 125 ms


## 3) Renommage des clés fonctionnelles

In [182]:
%%time
ts.rename(columns={'clientcode':'SessionID', 'coursecode':'FormationID', 'startdate':'SessionDate'},inplace=True)
tsv2.rename(columns={'clientcode':'SessionID'},inplace=True)
tr.rename(columns={'clientcode':'SessionID', 'traineusername':'USERNAME'},inplace=True)
tp.rename(columns={'plancode':'PlanID'},inplace=True)
emp.rename(columns={'username':'USERNAME'}, inplace=True)
tcw.rename(columns={'clientcode':'WishID','coursecode':'FormationID','username':'USERNAME'},inplace=True)
tiw.rename(columns={'username':'USERNAME', 'employeenumber':'ZY00.MATCLE', 'clientcode':'WishID', 'coursecode':'FormationID'},inplace=True)
tpcw.rename(columns={'plan_code':'PlanID', 'wish_code':'WishID'}, inplace=True)
tpci.rename(columns={'plan_code':'PlanID', 'wish_code':'WishID'}, inplace=True)

Wall time: 15.6 ms


## 4) Alimentation des dictionnaires contenant les colonnes et dataframe

In [183]:
%%time
#my_dict_columns = {"tcw": tcw.columns, "tiw": tiw.columns, "tp" : tp.columns, "tpcw" : tpcw.columns, "tr" : tr.columns, "ts" : ts.columns, "tsc" : tsc.columns, "tsv2" : tsv2.columns, "emp" : emp.columns,
 #          "empc" : empc.columns, "indO": indO.columns, "indpp": indpp.columns}

my_dict_dataframe = {"tcw": tcw, "tiw": tiw, "tp" : tp, "tpcw" : tpcw, "tr" : tr, "ts" : ts, "tsc" : tsc, "tsv2" : tsv2, "emp" : emp,
           "empc" : empc, "indO": indO, "indpp": indpp}

Wall time: 0 ns


## 5) Creation des tables de dimension

### Création de la table dimension demandes de formations

In [184]:
%%time
dim_wish = tcw.append(tiw,sort=True)
dim_wish.reset_index(drop=True, inplace=True)
dim_wish["Wish_key"] = dim_wish.index
dim_wish.loc[dim_wish.nbtrainees.isna(), "nbtrainees"] = 1
#dim_wish = df_mapp(dim_wish)
my_dict_dataframe["dim_wish_table"]= dim_wish
#wish_table.to_excel(r'Transformed data\wish_table.xlsx')

Wall time: 31.2 ms


### Traitement du mapping des colonnes et des valeurs

In [185]:
%%time
try:
    pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping dim colonne")
    Mapp_sheet = True    
except:
    Mapp_sheet = False
    
if Mapp_sheet == True:    
    pass
else:
    add_sheet_mapcol(my_dict_dataframe)

my_dict_dataframe = map_col(my_dict_dataframe)

print(my_dict_dataframe.keys())
mapp_data(my_dict_dataframe)

dict_keys(['tcw', 'tiw', 'tp', 'tpcw', 'tr', 'ts', 'tsc', 'tsv2', 'emp', 'empc', 'indO', 'indpp', 'dim_wish_table'])
[nan 'Medium' 'Important' 'Low']
['TS_DEVELOPPEMENT' 'TS_MANAGERREQUEST' "Campagne d'entretiens annuels"
 'Yearly appraisals' 'MY_TS' 'My Space' 'TS_EVALUATION' 'Bewertungsrunde'
 'Campagne de Recueil des besoins formation' 'TALENT_REVIEW'
 'Entretiens annuels' 'Fuera de campaña' 'Utvecklingssamtal'
 'Medarbetarsamtal' 'Utanför kampanj' 'Utvecklingssamtal 2013' 'MUS 2016'
 'Medarbejdersamtaler' 'MUS 2017' 'Entrevistas anuales'
 'TS|Campaign|Label|TR_Campaign' 'Mitarbeitergespräch 2019'
 'TS|Campaign|Label|PR']
Wall time: 437 ms


In [186]:
print(my_dict_dataframe["dim_wish_table"]["priorité"].unique())

[nan 'Medium' 'Important' 'Low']


## 6) Création des tables de fait

### Table de fait demande de formation

In [187]:
%%time
dim_wish = dim_wish[["Wish_key", "nbtrainees"]].copy()
dim_wish.loc[:, "Nb_demandes"]= 1

my_dict_dataframe["wish_fact"] = wish_fact

NameError: name 'wish_fact' is not defined

## Export des données vers le répertoire Transformed data

In [188]:
%%time
for key, value in my_dict_dataframe.items():
        location = "Transformed data"
        file_name = str(key) + '.csv'
        location = os.path.join(location, file_name)  
        value.to_csv(location, encoding='utf16', index=False)

Wall time: 624 ms
